<a href="https://colab.research.google.com/github/CianOSull/AutoML_With_SA_FYP_2021/blob/Preprocessing/MainNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Notebook for running all the libraries

How this notebook works is that it contains the code
for loading and cleaning the dataset.

Then there is multiple branches created on the
Github that include the code for running each library.

E.g. MLBox branch has the code for running MLBox.

# CURRENT BRANCH: [INSERT BRANCH NAME HERE]

# Install Library

In [ ]:
# Insert any install comamnds in this cell


# Preprocessing Section

In [ ]:
# Import the necessary modules for cleaning
import math
import time 
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Download the necessary parts for the NLTK module
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Create Model Section


# SKLearn Inbuilt tfidf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# A dense numpy array of the sparse matrix outputted can be made by using:
# X_train.toarray()
def create_data(num, process):  
  imdb_df = pd.read_csv("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/IMDB_Dataset.csv")

  # the next step is to randomize the rows of the data
  imdb_df = imdb_df.sample(frac=1).reset_index(drop=True)

  imdb_df['sentiment'] = imdb_df['sentiment'].map({'negative' : 0, 'positive' : 1})

  features = imdb_df.review.values[0:num]
  labels = imdb_df.sentiment.values[0:num]

  no_samples = 0.8

  # This gets the percentage of indexes from feature vector and uses those for training
  train_txt = features[0:int(no_samples*len(features))]
  y_train = labels[0:int(no_samples*len(labels))]

  # Go from the index that was used for training to the final
  test_txt = features[int(no_samples*len(features)):len(features)]
  y_test = labels[int(no_samples*len(labels)):len(labels)]

  # This cell has the sk learn functions
  # Allows you to choose which function you want to use
  if process == 0:
    ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

    ctv.fit(features)

    X_train_sparse = ctv.transform(train_txt)
    X_test_sparse = ctv.transform(test_txt)

  else:
    tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

    tfv.fit(features)

    X_train_sparse = tfv.transform(train_txt)
    X_test_sparse = tfv.transform(test_txt)

  # Converting to numpy arrays for more generic format
  X_train = X_train_sparse.toarray()
  X_test = X_test_sparse.toarray()

  return X_train, y_train, X_test, y_test

In [ ]:
# Create train test split
X_train, y_train, X_test, y_test = create_data(500, 1)
print(type(X_train))

# Run the automl



In [ ]:
# Insert AutoML code here


# Metrics

In [ ]:
# Create the variable y_pred that will contain the predictions


In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.75      0.74        52
           1       0.72      0.69      0.70        48

    accuracy                           0.72       100
   macro avg       0.72      0.72      0.72       100
weighted avg       0.72      0.72      0.72       100

[[39 13]
 [15 33]]


# Export The Models/Pipelines

In [ ]:
# Insert any code to export the models/pipelines created


# Test Dataset

This area is for running the test dataset.

In [ ]:
# A dense numpy array of the sparse matrix outputted can be made by using:
# X_train.toarray()
def create_test_data(process):  
  test_df = pd.read_csv("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/NPS_TestContent.csv", encoding='utf8', engine='c')
  
  # 2 null rows were found so drop them
  print(test_df.isnull().sum().sum())
  test_df = test_df.dropna()
  print(test_df.isnull().sum().sum())

  # the next step is to randomize the rows of the data
  test_df = test_df.sample(frac=1).reset_index(drop=True)

  test_df['sentiment'] = test_df['sentiment'].map({'Negative' : 0, 'Positive' : 1})

  features = test_df.test.values
  labels = test_df.sentiment.values

  no_samples = 0.8

  # This cell has the sk learn functions
  # Allows you to choose which function you want to use
  if process == 0:
    ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

    ctv.fit(features)

    features_sparse = ctv.transform(features)

  else:
    tfv = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

    tfv.fit(features)

    features_sparse = tfv.transform(features)

  # Converting to numpy arrays for more generic format
  features_t = features_sparse.toarray()

  return features_t, labels

In [ ]:
features, labels = create_test_data(1)

# Multiple column format
tfidf_df = pd.DataFrame(features)
tfidf_df['labels'] = labels

print(tfidf_df.head())

# Exported Pipeline/Models

In [ ]:
# Create train test split
X_train, y_train, X_test, y_test = create_data(500, 1)
print(type(X_train))

# Insert the calling of exported pipelines/models here


In [ ]:
from sklearn import metrics
print(metrics.classification_report(testing_target, results))
print(metrics.confusion_matrix(testing_target, results))